<a href="https://colab.research.google.com/github/swastikbanerjee/ML_Lab/blob/main/2348566_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import threading
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)
from mlxtend.frequent_patterns import apriori,fpgrowth

In [ ]:
data = pd.read_csv("/content/Groceries_dataset (1).csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(38765, 3)

In [ ]:
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [ ]:
data.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Member_number      0
Date               0
itemDescription    0
dtype: int64

In [ ]:
transactions=[]
for index,row in data.iterrows():
  transactions.append(row.dropna().tolist)
transactions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[<bound method IndexOpsMixin.tolist of Member_number                1808
 Date                   21-07-2015
 itemDescription    tropical fruit
 Name: 0, dtype: object>,
 <bound method IndexOpsMixin.tolist of Member_number            2552
 Date               05-01-2015
 itemDescription    whole milk
 Name: 1, dtype: object>,
 <bound method IndexOpsMixin.tolist of Member_number            2300
 Date               19-09-2015
 itemDescription     pip fruit
 Name: 2, dtype: object>,
 <bound method IndexOpsMixin.tolist of Member_number                  1187
 Date                     12-12-2015
 itemDescription    other vegetables
 Name: 3, dtype: object>,
 <bound method IndexOpsMixin.tolist of Member_number            3037
 Date               01-02-2015
 itemDescription    whole milk
 Name: 4, dtype: object>,
 <bound method IndexOpsMixin.tolist of Member_number            4941
 Date               14-02-2015
 itemDescription    rolls/buns
 Name: 5, dtype: object>,
 <bound method IndexOpsMixin

In [ ]:
# encoded=pd.get_dummies(pd.DataFrame(transactions).stack()).sum(level=0)
# transactionsdf=(encoded>0).astype(int)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd.DataFrame(transactions)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
0,<bound method IndexOpsMixin.tolist of Member_n...
1,<bound method IndexOpsMixin.tolist of Member_n...
2,<bound method IndexOpsMixin.tolist of Member_n...
3,<bound method IndexOpsMixin.tolist of Member_n...
4,<bound method IndexOpsMixin.tolist of Member_n...
...,...
38760,<bound method IndexOpsMixin.tolist of Member_n...
38761,<bound method IndexOpsMixin.tolist of Member_n...
38762,<bound method IndexOpsMixin.tolist of Member_n...
38763,<bound method IndexOpsMixin.tolist of Member_n...


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
